<a href="https://colab.research.google.com/github/minako-aino/crypto-FB-9/blob/cp3/cp3/dorosh_fb92_shatkovska_fb92_cp3/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторна робота з криптографії №3
#### Виконали: Дорош Анастасія та Шатковська Діана ФБ-92


**Мета:** засвоєння понять ентропії на символ джерела та його надлишковості, вивчення та порівняння різних моделей джерела відкритого тексту для наближеного визначення
ентропії, набуття практичних навичок щодо оцінки ентропії на символ джерела.



###Хід роботи

Say smth

###Частина 1


Реалізувати підпрограми із необхідними математичними операціями:
обчисленням оберненого елементу за модулем із використанням розширеного алгоритму Евкліда, розв’язуванням лінійних порівнянь. При розв’язуванні порівнянь потрібно коректно обробляти випадок із декількома розв’язками, повертаючи їх усі.


In [159]:
def gcd(a, b):
  p = [0,1]
  gcd_val = b
  a, b = max(a,b), min(a,b)
  while (b != 0):
    # print('---')
    # print(a,b)
    q = a // b
    # print(q)
    gcd_val = b
    a, b = b, a % b
    p.append(p[-1]*(-q)+p[-2])
    # print(p[-1])
    # gcd_val = gcd, p[-2] = a^(-1)
  return gcd_val, p[-2]

# debug
gcd(13, 151)

(1, -58)

In [160]:
def linear(a, b, n):
  a = a%n
  b = b%n
  
  d, a_re = gcd(a, n)

  if d == 1:
    x = (a_re*b) % n

    return [x]
  else:
    if b%d == 0:
      solutions = []
      solutions_loc = []

      a1 = a/d
      b1 = b/d
      n1 = n/d

      solutions_loc = linear(a1, b1, n1)

      for i in range(0, d):
        for x0 in solutions_loc:
          res = x0+(d-i)*n1
          solutions.append(res%n)

      #print(f"There are {d} solutions: ")
      return sorted(solutions)
    else:
      #print("No solutions")
      return None

In [161]:
# test
linear(154,22,803)

[21.0, 94.0, 167.0, 240.0, 313.0, 386.0, 459.0, 532.0, 605.0, 678.0, 751.0]

###Частина 2

За допомогою програми обчислення частот біграм, яка написана в ході виконання комп’ютерного практикуму No1, знайти 5 найчастіших біграм запропонованого шифртексту (за варіантом).

In [162]:
# stuff to get the file
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [163]:
cd /content/drive/MyDrive/5 sem/Crypto/lab3

/content/drive/MyDrive/5 sem/Crypto/lab3


In [164]:
ls

01_clean.txt  01_utf8.txt            for_tests_V1  test1.txt
01.txt        for_tests_utf8_V1.txt  lab3.gdoc     test_clean.txt


In [165]:
def clean_text(txt):
    with open(txt, 'r', encoding='utf-8') as file:
        text = file.read().lower()

    chars = '.71()-«5d?[“!93286”…—4;»0:],naoti*IVXvxse–'
    for ch in chars:
        text = text.replace(ch, '')
        
    text = '_'.join([word.strip('\n') for word in text.split()])
    text = ''.join([word.strip('\n') for word in text.split('_')])

    with open('01_clean.txt', 'w', encoding='utf-8') as file:
        file.write(text)

#clean_text("for_tests_utf8_V1.txt")
clean_text("01_utf8.txt")

In [166]:
def open_file(txt):
    with open(txt, 'r', encoding='utf-8') as file:
        text = file.read().lower()
    return text

test_raw = open_file("for_tests_utf8_V1.txt")
text = open_file("01_clean.txt")
test = open_file("test_clean.txt")

In [167]:
from collections import Counter
import operator
from itertools import islice

# from lab1
def count_bi_nointersect(text):
    res = Counter(text[idx: idx + 2] for idx in range(0, (len(text)), 2))
    total_bi = sum(res.values())
    res = {x: round(res[x]/total_bi, 10) for x in res}
    return dict(res)
    
main_count = count_bi_nointersect(test)

# sort
test_count_sorted = dict(sorted(test_count.items(), key=operator.itemgetter(1),reverse=True))

# get the first 5 items
n_items = dict(islice(test_count_sorted.items(), 5))
print(n_items)

{'вэ': 0.0221562809, 'кь': 0.0172106825, 'ыу': 0.0168150346, 'шк': 0.0152324431, 'ди': 0.0130563798}


###Частина 3

Перебрати можливі варіанти співставлення частих біграм мови та частих біграм шифртексту (розглядаючи пари біграм із п’яти найчастіших). Для кожного співставлення знайти можливі кандидати на ключ (a,b) шляхом розв’язання системи (1).

In [168]:
alphabet_full = 'абвгдежзийклмнопрстуфхцчшщьыэюя'
alphabet = list(alphabet_full)
top_bi = ('ст', 'но', 'то', 'на', 'ен')

 # determine bigram's value
def bi_val(bi):
  val = alphabet.index(bi[0])*len(alphabet) + alphabet.index(bi[1])
  return val

 # getting all possible pairs of bigrams cyphered-real
def all_pairs(cyphered, real):
  res = []
  for el1 in cyphered:
    for el2 in real:
      res.append((el1, el2))
  
  res2 = []
  for p1 in res:
    for p2 in res:
      if p2[0] not in p1 and p2[1] not in p1:
        res2.append((p1,p2))
  return res2

 # finding mutual key from a pair of bigrams cyphered-real
def find_key(pair1, pair2):  # (y1, x1) (y2, x2)
  y1, x1 = bi_val(pair1[0]), bi_val(pair1[1])
  y2, x2 = bi_val(pair2[0]), bi_val(pair2[1])

  a = linear(y1-y2,x1-x2,len(alphabet)**2)
  b = []
  if a != None:
    for el in a:
      b.append((y1-(x1*el))%len(alphabet)**2)
  
    return a,b

In [169]:
 # testing the code above
n_items = {'вэ': 0.0221562809, 'кь': 0.0172106825, 'ыу': 0.0168150346, 'шк': 0.0152324431, 'ди': 0.0130563798}
allp = all_pairs(n_items.keys(),top_bi)[0]
print(allp)
#print(bi_val(allp[0][0]))
#print(bi_val(allp[0][1]))
#print(bi_val(allp[1][0]))
#print(bi_val(allp[1][1]))
#print(linear(714,128,961))
key_a, key_b = find_key(allp[0], allp[1])
print(key_a, "\t", key_b)

(('вэ', 'ст'), ('кь', 'но'))
[312] 	 [147]


###Частина 4

Для кожного кандидата на ключ дешифрувати шифртекст. Якщо шифртекст не є змістовним текстом російською мовою, відкинути цього кандидата.

In [170]:
def decode(a, b, text):
  plaintext = ''
  n = len(alphabet)
  # get inverse of a
  a_inv = gcd(a, n**2)[1]

  # go through bigrams
  for bi in [text[idx: idx + 2] for idx in range(0, (len(text)), 2)]:
    bi_idx = bi_val(bi)
    deciph_bi = ((bi_idx - b) * a_inv) % (n**2)

    x2_idx = deciph_bi % n
    x1_idx = (deciph_bi - x2_idx) / n

    x1 = alphabet[int(x1_idx)]
    x2 = alphabet[int(x2_idx)]
    
    plaintext += x1 + x2

  return plaintext

In [177]:
def avg(data):
  return sum(data.values()) / len(data)

# may be refactored later
def find_sence(text):
    forbidden_lst = ('аъ', 'аь', 'бй', 'бф', 'гщ', 'гъ', 'еъ', 'еь', 'жй', 'жц', 'жщ', 'жъ', 'жы', 'йъ', 'къ', 'лъ', 'мъ', 'оъ', 'пъ', 'ръ', 'уъ','уь', 'фщ', 'фъ', 'хы', 'хь', 'цщ', 'цъ', 'цю', 'чф', 'чц', 'чщ', 'чъ', 'чы', 'чю', 'шщ', 'шъ', 'шы', 'шю', 'щг', 'щж','щл', 'щх', 'щц', 'щч', 'щш', 'щъ', 'щы', 'щю', 'щя', 'ъа', 'ъб', 'ъг', 'ъд', 'ъз', 'ъй', 'ък', 'ъл', 'ън', 'ъо', 'ъп', 'ър','ъс', 'ът', 'ъу', 'ъф', 'ъх', 'ъц', 'ъч', 'ъш', 'ъщ', 'ъъ', 'ъы', 'ъь', 'ъэ', 'ыъ', 'ыь', 'ьъ', 'ьы', 'эа', 'эж', 'эи', 'эо','эу', 'эщ', 'эъ', 'эы', 'эь', 'эю', 'эя', 'юъ', 'юы', 'юь', 'яъ', 'яы', 'яь', 'ьь')
    current_lst = count_bi_nointersect(text)
    result = 0
    counter = 0
    
    for f in forbidden_lst:
        if f in current_lst:
            result += current_lst[f]
            counter += 1
    try:
        final = result/counter
    except ZeroDivisionError:
        return True
    # may be refactored
    if final <= avg(current_lst):
        return True

###Частина 5

Повторювати дії 3-4 доти, доки дешифрований текст не буде змістовним.

In [184]:
results = []
def try_pair(pair):
  #try:
    keys_a, keys_b = find_key(pair[0], pair[1])
    for key_a, key_b in zip(keys_a, keys_b):
      text = decode(key_a, key_b, test)
      # if find_sence(text):
      final = (key_a, key_b)
      f = open("demofile2.txt", "a")
      f.write(str(final))
      f.write(text)
      f.close()
      #print("final: ", final)
      #print(text)
      return 1
  #except TypeError:
    #print("An error occured")

In [185]:
status = False
allp = all_pairs(n_items.keys(),top_bi)
for pair in allp:
  try_pair(pair)
#final = decode(13, 151, text)
#print(final)
    #if try_pair():
    #  break

###Висновки

Say smth